In [1]:
import os
os.getcwd()

'c:\\Users\\Marina\\Desktop\\cicd-project\\src\\components'

In [3]:
os.chdir("../../")
os.getcwd()

'c:\\Users\\Marina\\Desktop\\cicd-project'

In [7]:
import os
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

import psycopg2
from psycopg2.extras import RealDictCursor


DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_ENDPOINT')
DB_PORT = os.getenv('DB_PORT')

try:
    conn = psycopg2.connect(
        # database= DB_NAME,
        user = DB_USER,
        password = DB_PASSWORD,
        host = DB_HOST,
        port = DB_PORT
    )
    conn.autocommit = True
    cur = conn.cursor(cursor_factory=RealDictCursor)
    cur.execute

    print(f"Connected Successfully")
except Exception as e:
    print(e)

Connected Successfully


In [12]:
table_create_query = "CREATE TABLE customerInfo(id SERIAL PRIMARY KEY, name TEXT, age FLOAT)"


In [13]:
cur.execute(table_create_query)

In [14]:
insert_query = """insert into customerInfo (name, age) values ('abc', 22)"""

In [15]:
cur.execute(insert_query)

#### aa

In [18]:
from sqlalchemy.orm import sessionmaker, declarative_base
Base = declarative_base()

from sqlalchemy import create_engine, Column, Integer, String, DateTime, Float


"""
https://claude.ai/chat/1c3c494d-eda2-465a-8f20-d466a8320fd1
Error logging predictions to database: connection to server at "prediction-logs-db.czgceia6cqs7.us-east-2.rds.amazonaws.com" (3.18.205.193), port 5432 failed: FATAL: no pg_hba.conf entry for host "177.11.78.212", user "postgres", database "prediction-logs-db", no encryption Logging failed: connection to server at "prediction-logs-db.czgceia6cqs7.us-east-2.rds.amazonaws.com" (3.18.205.193), port 5432 failed: FATAL: no pg_hba.conf entry for host "177.11.78.212", user "postgres", database "prediction-logs-db", no encryption
"""
class PredictionLog(Base):
    __tablename__ = 'prediction_logs'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    timestamp = Column(DateTime, default=datetime.utcnow)
    gender = Column(String)
    race_ethnicity = Column(String)
    parental_level_of_education = Column(String)
    lunch = Column(String)
    test_preparation_course = Column(String)
    reading_score = Column(Float)
    writing_score = Column(Float)
    prediction = Column(Float)

# TODO: for production, include the SSL verification
class DatabasePredictionLogger:
    """
    Logger for storing predictions in a PostgreSQL database
    """
    def __init__(self):
        """
        Initialize database connection parameters
        """
        # Database connection parameters from environment variables
        self.db_endpoint = os.getenv("DB_ENDPOINT")
        self.db_port = os.getenv("DB_PORT")
        self.db_name = os.getenv("DB_NAME")
        self.db_user = os.getenv("DB_USER")
        self.db_password = os.getenv("DB_PASSWORD")

    def _get_connection(self):
        """
        Create and return a database connection
        
        Returns:
            psycopg2.extensions.connection: Database connection
        """
        try:
            conn = psycopg2.connect(
                user=self.db_user,
                password=self.db_password,
                host=self.db_endpoint,
                port=self.db_port,
                cursor_factory=RealDictCursor  # Use RealDictCursor by default
            )
            conn.autocommit = True  # Set autocommit to True
            return conn
        except psycopg2.Error as e:
            print(f"Error connecting to the database: {e}")
            raise

    def log_prediction(self, input_data, prediction):
        """
        Log a single or batch prediction to the PostgreSQL database
        
        Args:
            input_data: Input feature(s)
            prediction: Prediction result(s)
        
        Returns:
            int: Number of logs inserted
        """
        # Ensure inputs are lists (assuming you have PredictionInput and PredictionOutput classes)
        if not isinstance(input_data, list):
            input_data = [input_data]
        
        if not isinstance(prediction, list):
            prediction = [prediction]
        
        # Validate input lengths
        if len(input_data) != len(prediction):
            raise ValueError("Input data and prediction lists must have the same length")

        conn = None
        try:
            conn = self._get_connection()
            
            with conn.cursor() as cur:
                # Prepare insert statement
                insert_query = """
                INSERT INTO prediction_logs (
                    timestamp, gender, race_ethnicity, 
                    parental_level_of_education, lunch, 
                    test_preparation_course, reading_score, 
                    writing_score, prediction
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                """
                
                # Prepare log entries
                log_entries = []
                for inp, pred in zip(input_data, prediction):
                    log_entry = (
                        datetime.utcnow(),
                        inp.gender,
                        inp.race_ethnicity,
                        inp.parental_level_of_education,
                        inp.lunch,
                        inp.test_preparation_course,
                        inp.reading_score,
                        inp.writing_score,
                        pred.prediction
                    )
                    log_entries.append(log_entry)
                
                # Execute batch insert
                cur.executemany(insert_query, log_entries)
            
            print(f"Successfully logged {len(log_entries)} prediction(s) to database")
            return len(log_entries)
        
        except psycopg2.Error as e:
            print(f"Database error logging predictions: {e}")
            raise
        finally:
            if conn:
                conn.close()

    def create_prediction_logs_table(self):
        """
        Create the prediction_logs table if it doesn't exist
        """
        conn = None
        try:
            conn = self._get_connection()
            
            with conn.cursor() as cur:
                # Create table if not exists
                create_table_query = """
                CREATE TABLE IF NOT EXISTS prediction_logs (
                    id SERIAL PRIMARY KEY,
                    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    gender VARCHAR(50),
                    race_ethnicity VARCHAR(50),
                    parental_level_of_education VARCHAR(100),
                    lunch VARCHAR(50),
                    test_preparation_course VARCHAR(100),
                    reading_score FLOAT,
                    writing_score FLOAT,
                    prediction FLOAT
                )
                """
                
                # Execute create table query
                cur.execute(create_table_query)
            
            print("Prediction logs table ensured.")
        
        except psycopg2.Error as e:
            print(f"Error creating prediction logs table: {e}")
            raise
        finally:
            if conn:
                conn.close()



In [20]:
DatabasePredictionLogger().create_prediction_logs_table()

Prediction logs table ensured.
